In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import logging
import imp
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri

RANDOM_SEED = 0
numpy2ri.activate()

import fairtest.utils.log as fairtest_log
imp.reload(fairtest_log)
fairtest_log.set_params(filename='fairtest.log', level=logging.INFO)

In [28]:
import fairtest.investigation as inv
import fairtest.testing as testing
import fairtest.discovery as discovery
import fairtest.error_profiling as error_profiling
import fairtest.modules.metrics as metrics
import fairtest.modules.metrics.correlation as correlation
import fairtest.modules.metrics.regression as regression
import fairtest.modules.metrics.binary_metrics as binary_metrics
import fairtest.modules.statistics.confidence_interval as intervals
import ast
import sklearn.preprocessing as preprocessing
'''
imp.reload(intervals)
imp.reload(inv)
imp.reload(testing)
imp.reload(discovery)
imp.reload(error_profiling)
imp.reload(metrics)
imp.reload(correlation)
imp.reload(binary_metrics)
imp.reload(regression)
'''

'''
dataname = 'adult'
data = pd.read_csv(
            '../data/adult/adult.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('fnlwgt', axis=1)
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Indian' if 'Indian' in x else x, data['race'])
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Black & Other' if ('Indian' in x or 'Other' in x or 'Black' in x) else x, data['race'])
TARGET = 'income'
SENS = ['sex', 'race']
EXPL = []
'''

'''
dataname = 'medical_bin'
data = pd.read_csv(
            '../data/medical/predictions_bin.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''
'''
dataname = 'medical_reg'
data = pd.read_csv(
            '../data/medical/predictions_reg_log.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''
'''
dataname = 'berkeley'
data = pd.read_csv(
            '../data/berkeley/berkeley.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'accepted'
SENS = ['gender']
EXPL = ['department']
EXPL = []
'''
'''
dataname = 'staples'
data = pd.read_csv(
            '../data/staples/staples.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('zipcode', axis=1)
data = data.drop('distance', axis=1)
data = data.drop('city', axis=1)
TARGET = 'price'
SENS = ['income', 'race']
EXPL = []
'''
'''
data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 
                   'Indian & Alaska Native' if 'Indian' in x  else
                   'Black' if 'Black' in x  else
                   'Pacific Islander' if 'Pacific' in x else
                   'White' if 'White' in x else
                   'Hispanic' if 'Latino' in x else
                   '2+ Races' if 'Two' in x else
                   'Other' if 'Other' in x else
                    x, data['race'])
'''

dataname = 'overfeat'
data = pd.read_csv(
            '../data/images/overfeat_raw.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
TARGET = labels.tolist()

'''
dataname = 'caffe'
data = pd.read_csv(
            '../data/images/caffe.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
TARGET = labels.tolist()
'''
'''
dataname = 'benchmark'
data = pd.read_csv(
            '../data/benchmark/benchmark.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'price'
SENS = ['race', 'income']
EXPL = []
'''

#metrics = {'gender': 'Diff'}
metrics = {}

#exp = testing.Testing(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
#exp = error_profiling.ErrorProfiling(data, SENS, TARGET, GROUND_TRUTH, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
exp = discovery.Discovery(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, topk=35, random_state=RANDOM_SEED)
print len(exp.train_set), len(exp.test_set)
exp.train_set.tail()

INFO:root:New Discovery Investigation
INFO:root:Encoding Feature Race
INFO:root:Target Feature: Target(names=[Afghan hound ... zucchini], arity=None)
INFO:root:Training Size 1323 -  Testing Size 1324
INFO:root:Choosing metric REGRESSION for feature Race


1323 1324


,Race,Afghan hound,African elephant,African grey,American Staffordshire terrier,American lobster,Arabian camel,Band Aid,Bedlington terrier,Bernese mountain dog,...,wire-haired fox terrier,wok,wombat,wooden spoon,wool,worm fence,wreck,yurt,zebra,zucchini
1731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
763,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
835,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
import fairtest.modules.context_discovery.guided_tree as guided_tree
#imp.reload(guided_tree)
inv.train([exp], score_aggregation='avg', min_leaf_size=100, max_depth=5)

INFO:root:Begin training phase with protected feature Race
INFO:root:Building a Guided Decision Tree


In [17]:
print exp.trained_trees['Age']

KeyError: 'Age'

In [30]:
import fairtest.modules.statistics.hypothesis_test as tests
import fairtest.modules.statistics.multiple_testing as multitest
import fairtest.modules.context_discovery.tree_parser as tree_parser
import fairtest.modules.statistics.confidence_interval as intervals
#imp.reload(tests)
#imp.reload(multitest)
#imp.reload(tree_parser)
#imp.reload(intervals)

inv.test([exp], exact=True, prune_insignificant=True)

INFO:root:Begin testing phase
INFO:root:Testing 35 hypotheses


In [31]:
import fairtest.modules.bug_report.report as rep
import fairtest.modules.bug_report.filter_rank as fr
imp.reload(rep)
imp.reload(fr)

output_dir = None
inv.report([exp], dataname, output_dir=output_dir, filter_conf=0.95, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)

Commit Hash: 	b9294718751f4007e53622c8d7845dcfddab22ff

Dataset: overfeat
Train Size: 1323
Test Size: 1324
S: ['Race']
X: []
E: None
O: [Afghan hound ... zucchini]

Train Params: 	{'max_bins': 10, 'min_leaf_size': 100, 'max_depth': 5, 'agg_type': 'avg'}
Test Params: 	{'prune_insignificant': True, 'exact': True, 'family_conf': 0.95}
Report Params: 	{'node_filter': 'better_than_ancestors'}

Report of associations of O=[Afghan hound ... zucchini] on Si = Race:
Association metric: REG(topk=35)

Global Population 0 of size 1324

Average Effect of top-35 labels: 0.0116549403528

                      ci_low   ci_high          pval
maillot            -0.230576 -0.115685  2.527311e-20
brassiere          -0.178185 -0.079587  2.596893e-15
miniskirt          -0.136330 -0.047248  1.644527e-09
lipstick           -0.119964 -0.039109  1.128632e-08
ski mask            0.022639  0.078864  2.657107e-07
gown               -0.096003 -0.026046  7.944168e-07
mask                0.027526  0.101763  8.116713e

[{}]